# Normality Tests

## View some data

In [1]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from constants.real_time_constants import SLIDING_WINDOW_LEN, STRIDE
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset
from data.sliding_window import SlidingWindowDataset

## pure data
id_train_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_dataset"
)
id_test_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_dataset"
)
ood_test_base_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_dataset"
)

## dataset that takes chunks of 120 tokens out of the above datasets
id_train_dataset = SlidingWindowDataset(
    base_dataset=id_train_base_dataset,
    name="id_train_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)
id_test_dataset = SlidingWindowDataset(
    base_dataset=id_test_base_dataset,
    name="id_test_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)   
ood_test_dataset = SlidingWindowDataset(
    base_dataset=ood_test_base_dataset,
    name="ood_test_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)




/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Detected 0 bad samples
Detected 0 bad samples


In [2]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    try:
        events = get_readable_events(input_ids)
    except Exception as e:
        print(f"Error: {e}")
        print(f"Input IDs: {input_ids}")
        continue
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 50)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic", "vel"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated",
    "vel": "velocity"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df.to_csv("events.csv", index=False)
df

dataset id_train                                   id_test              ...  \
key         misc onset   dur inst pitch  antic vel    misc onset   dur  ...   
0             AR                                        AR              ...   
1                  4.1  0.39    0    D4  False              1.28  0.07  ...   
2                  4.1  0.31    0    A4  False              1.38  0.06  ...   
3                  4.1  0.16    0    E4  False              1.39  0.08  ...   
4                 4.11  0.16    0    E5  False              1.48  0.08  ...   
5                 4.14  0.03    0    D5  False              1.49  0.09  ...   
6                 4.43  0.14    0    D3  False              1.59  0.08  ...   
7                 4.43  0.09    0    F4  False              1.64  0.05  ...   
8                 4.43  0.14    0    F5  False              1.68  0.06  ...   
9                 4.73  0.07    0    A5  False              1.69  0.04  ...   
10                4.73  0.07    0    A4  False              1.75  0.09  ...   
11                4.89   0.1    0    E3  False               1.8  0.04  ...   
12                5.07  0.33    0    C5  False               1.9  0.08  ...   
13                5.07  0.18    0    E4  False               1.9  0.09  ...   
14                5.07  0.97    0    E5  False              1.99  0.06  ...   
15                5.08  0.32    0    A4  False              2.02  0.05  ...   
16                5.08  0.35    0    E3  False              2.12  0.07  ...   
17                5.08  0.41    0    A3  False              2.12   0.0  ...   
18                5.08  0.38    0    A2  False              2.21  0.19  ...   
19                5.38  0.22    0    E4  False              2.23  0.19  ...   
20                5.54  0.55    0    C5  False              2.43  0.06  ...   
21                5.71  0.36    0    A4  False              2.44  0.06  ...   
22                5.86   0.4    0    B3  False              2.53  0.12  ...   
23                6.02  0.32    0    C4  False              2.54  0.12  ...   
24                6.16  0.12    0    E4  False              2.65  0.09  ...   
25                6.32  0.04    0    A3  False              2.66  0.06  ...   
26                6.48  0.12    0    A3  False              2.87   0.1  ...   
27                6.64  0.47    0    C4  False              2.88  0.07  ...   
28                 6.8  0.55    0    E4  False              2.97   0.1  ...   
29                 6.8  0.52    0    A3  False              2.98  0.07  ...   
30                6.96  0.08    0    E3  False              3.06  0.06  ...   
31                7.11  0.22    0    A4  False              3.08  0.07  ...   
32                7.26  0.08    0    C4  False              3.18  0.09  ...   
33                7.43  0.07    0    A2  False              3.18  0.08  ...   
34                 7.6  0.36    0    D4  False              3.27   0.1  ...   
35                 7.6  0.38    0    D3  False              3.29   0.1  ...   
36                 7.6  0.19    0    A3  False              3.39  0.05  ...   
37                 7.6  0.37    0    A4  False              3.39  0.08  ...   
38                 7.6  0.34    0    D2  False              3.45  0.15  ...   
39                7.61  0.26    0    A2  False              3.49  0.06  ...   
40                7.92  0.18    0    A3  False              3.58   0.1  ...   

dataset                  ood_test                                     
key     pitch  antic vel     misc  onset   dur inst pitch  antic vel  
0                              AR                                     
1          C4  False                12.5  0.03    0    E5  False      
2          D2  False               12.51  0.04    0    E3  False      
3          D4  False               12.51  0.03    0    E4  False      
4          G2  False               12.63  0.03    0    D4  False      
5          G4  False               12.64  0.04    0    D3  False      
6          D2  False              

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value <0.001
{'skewness': np.float64(171293.952), 'kurtosis': np.float64(-0.036), 'skewness_p': 0.756, 'kurtosis_p': 0.971, 'royston_p': 0.764, 'hz_p_value': 0.001}


### Extract layers

In [5]:
from extract_layers.extract_layers_main import extract_representations, collate_fn
from constants.model_constants import JORDAN_MODEL_NAME, DEVICE
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(JORDAN_MODEL_NAME).to(DEVICE)
num_layers = 24

id_train_dataloader = DataLoader(
    id_train_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn
)

extract_representations(
    model=model,
    data=id_train_dataloader,
    pooling_function=pooling_function,
    layers=list(range(num_layers + 1)),
)

Representations already exist. Loading from disk.


Loading layers from disk:  32%|███▏      | 8/25 [00:00<00:00, 73.31it/s]

Loading layer 0 from disk.
Loading layer 1 from disk.
Loading layer 2 from disk.
Loading layer 3 from disk.
Loading layer 4 from disk.
Loading layer 5 from disk.
Loading layer 6 from disk.
Loading layer 7 from disk.
Loading layer 8 from disk.
Loading layer 9 from disk.
Loading layer 10 from disk.
Loading layer 11 from disk.
Loading layer 12 from disk.
Loading layer 13 from disk.
Loading layer 14 from disk.


Loading layers from disk:  64%|██████▍   | 16/25 [00:00<00:00, 73.26it/s]

Loading layer 15 from disk.
Loading layer 16 from disk.
Loading layer 17 from disk.
Loading layer 18 from disk.
Loading layer 19 from disk.
Loading layer 20 from disk.
Loading layer 21 from disk.
Loading layer 22 from disk.


Loading layers from disk: 100%|██████████| 25/25 [00:00<00:00, 37.63it/s]

Loading layer 23 from disk.
Loading layer 24 from disk.


{0: array([[ 1.3061456e-03, -5.5287159e-03,  1.6389258e-04, ...,
          3.9908413e-02,  3.7244990e-02,  3.7832629e-02],
        [ 3.6730662e-03, -4.3182061e-03, -2.2318992e-03, ...,
          4.0613018e-02,  3.6641758e-02,  3.7586063e-02],
        [ 2.4871235e-03, -4.2648567e-03, -6.5752538e-05, ...,
          3.9956752e-02,  3.4127507e-02,  3.9603893e-02],
        ...,
        [ 9.7120823e-03,  1.6601959e-03,  2.2332061e-03, ...,
          3.8667031e-02,  3.8647264e-02,  3.6553409e-02],
        [ 1.0430695e-02,  3.3772897e-04,  1.4238092e-03, ...,
          3.6144767e-02,  3.5247546e-02,  3.5495654e-02],
        [ 8.4997043e-03,  2.1871249e-03,  4.1349838e-03, ...,
          3.6832038e-02,  3.1473108e-02,  3.5035428e-02]],
       shape=(2712, 2048), dtype=float32),
 1: array([[ 0.02728574, -0.0221916 , -0.0305368 , ...,  0.09844504,
          0.09787255,  0.12200287],
        [ 0.03178636, -0.00812103, -0.04976152, ...,  0.0913801 ,
          0.09227206,  0.10163207],
        [ 0.0

In [6]:
import numpy as np
import pandas as pd
from constants.file_format import get_extract_layers_file_path
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_dir = get_extract_layers_file_path(
        dataset_name="id_train_dataset",
        pooling_function_name=pooling_function.__name__,
        layer_idx=layer_idx,
    )
    layer_data = np.load(layer_dir)
    layer_data=layer_data[..., :10]
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (2712, 10)
Running test mardia
Running test royston
  Subsampling to 1000 samples and 10 features for testing
Running test hz


Processing layers:   4%|▍         | 1/25 [00:01<00:35,  1.50s/it]

p_value <0.001
Processing layer 1
Layer data shape: (2712, 10)
Running test mardia
Running test royston
  Subsampling to 1000 samples and 10 features for testing
Running test hz


Processing layers:   8%|▊         | 2/25 [00:03<00:37,  1.62s/it]

...

p_value <0.001
Processing layer 23
Layer data shape: (2712, 10)
Running test mardia
Running test royston
  Subsampling to 1000 samples and 10 features for testing
Running test hz


Processing layers:  96%|█████████▌| 24/25 [00:39<00:01,  1.60s/it]

p_value <0.001
Processing layer 24
Layer data shape: (2712, 10)
Running test mardia
Running test royston
  Subsampling to 1000 samples and 10 features for testing
Running test hz


Processing layers: 100%|██████████| 25/25 [00:41<00:00,  1.65s/it]

p_value <0.001


In [7]:
all_metrics

,skewness,kurtosis,skewness_p,kurtosis_p,royston_p,hz_p_value
0,15481.744,99.969,0.001,0.001,0.001,0.001
1,2514.134,12.570,0.001,0.001,0.001,0.001
2,2440.369,13.295,0.001,0.001,0.001,0.001
3,2137.920,11.774,0.001,0.001,0.001,0.001
4,2424.018,13.538,0.001,0.001,0.001,0.001
5,2071.396,9.224,0.001,0.001,0.001,0.001
6,2367.395,13.937,0.001,0.001,0.001,0.001
7,2327.672,12.383,0.001,0.001,0.001,0.001
8,1938.801,10.976,0.001,0.001,0.001,0.001
9,1750.985,8.484,0.001,0.001,0.001,0.001


In [8]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)